# 03. DeepStream with Vision model

이 강의에서는 ONNX 파일을 TensorRT 엔진 파일로 변환하여 DeepStream 애플리케이션에 배포하는 과정을 안내합니다. TensorRT는 딥러닝 모델의 추론을 최적화하여 고성능, 저지연 결과를 제공합니다.

## 03-2. TensorRT와 ONNX 소개
- **TensorRT**: NVIDIA에서 개발한 고성능 딥러닝 추론 최적화 및 런타임 라이브러리.
- **ONNX (Open Neural Network Exchange)**: 다양한 프레임워크 간 상호 운용성을 지원하는 딥러닝 모델 표현 형식.

### TensorRT로 변화하는 이유는? ###
- GPU 활용도를 극대화합니다.
- 모델 추론 지연 시간을 줄입니다.
- 추론을 위한 메모리 사용을 최적화합니다.

---

# 실습

## YOLOv8 모델로 engin파일 만들기

### 하드웨어 및 소프트웨어 요구 사항
- TensorRT를 지원하는 NVIDIA GPU.
- TensorRT SDK 설치.
- Python 환경 (Python 3.7 이상).
- 학습된 모델의 ONNX 파일.
- DeepStream 설치(테스트 용도, 선택 사항).

### **라이브러리 및 의존성**
Python 환경에서 다음 명령어로 필요한 라이브러리를 설치합니다:
```bash
pip install onnx onnxruntime tensorrt pycuda
```

### TensorRT 빌더 스크립트 준비

TensorRT는 ONNX 모델을 엔진 파일로 변환하는 도구를 제공합니다. 아래는 Python 기반의 접근 방식입니다.

In [ ]:
import tensorrt as trt

# TensorRT 로거
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

def build_engine(onnx_file_path, engine_file_path):
    # 빌더, 네트워크, 설정 생성
    with trt.Builder(TRT_LOGGER) as builder, \
         builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)) as network, \
         trt.OnnxParser(network, TRT_LOGGER) as parser:

        # 빌더 구성
        builder.max_batch_size = 1
        config = builder.create_builder_config()
        config.max_workspace_size = 1 << 30  # 1 GB

        # ONNX 파일 파싱
        with open(onnx_file_path, 'rb') as model:
            if not parser.parse(model.read()):
                print("ONNX 파일 파싱에 실패했습니다.")
                for error in range(parser.num_errors):
                    print(parser.get_error(error))
                return None

        # 엔진 빌드 및 직렬화
        engine = builder.build_engine(network, config)
        with open(engine_file_path, "wb") as f:
            f.write(engine.serialize())
        print(f"엔진 파일이 {engine_file_path}에 저장되었습니다.")

# 예제 사용법
build_engine("model.onnx", "model.engine")

### 엔진 파일 테스트

`.engine` 파일이 생성되면, 올바르게 동작하는지 테스트합니다.

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
import tensorrt as trt

# TensorRT 엔진 로드
def load_engine(engine_file_path):
    with open(engine_file_path, "rb") as f:
        runtime = trt.Runtime(TRT_LOGGER)
        return runtime.deserialize_cuda_engine(f.read())

engine = load_engine("model.engine")
print("엔진이 성공적으로 로드되었습니다.")

### DeepStream에서 활용
1. **엔진 파일 배치**: `.engine` 파일을 원하는 디렉터리로 이동합니다.
2. **DeepStream 구성 업데이트**: `config_infer_primary.txt` 파일을 수정하여 새 엔진 파일을 사용하도록 설정합니다.

예제:
```txt
[property]
model-engine-file=model.engine
labelfile-path=labels.txt
batch-size=1
```